# Binary to numpy
## Imports

In [5]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Imports
import traceback
import numpy as np
import warnings
import matplotlib.pyplot as plt    
import ipywidgets as widgets
import matplotlib.image as mpimg
import numpy.fft as fft
import threading
import time
import types
import scipy.signal
import signal
import sh
import sys
import os
import re
import struct
import visa
import logging
import copy
import glob

sys.path.append('/home/colindv/sciebo/masterthesis/remote control/Remote control v3 (version control)/')
home = os.environ['HOME']+'/' # home dir

from colors import color_array
from helper_functions import read_binary
from helper_functions import info, debug, warning, error, critical
from ContextManager import ContextManager
from Receiver import Receiver

qnap = "/storage/qnap/"
folder = "interference_free/ota/no_cfo/"
bin_folder = "bin/"

mod_schemes = ["SC_BPSK", "SC_QPSK", "SC_16QAM", "SC_64QAM",
               "OFDM_BPSK", "OFDM_QPSK", "OFDM_16QAM", "OFDM_64QAM"]

dummyReceiver = Receiver(**{'bin_dir': '/home/colindv/bin_data/', 
                            'freq': 5750000000.0, 
                            'srate': 50000000.0, 
                            'rx_dir': '/home/colindv/bin_data/', 
                            'bw_noise': 5000000.0, 
                            'bw': 100000.0, 
                            'vbw': 10000.0, 
                            'init_gain': 0, 
                            'freq_noise': np.array([5.73e9, 5.77e9]),
                            'span': 50000000.0, 
                            'max_gain': 0, 
                            'freq_noise_offset': 20000000.0, 
                            'bw_signal': 20000000.0})

dummyReceiver.name = "dummy"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## process binary files to numpy matrix

In [ ]:
storage_dir = "/storage/qnap/"
root_folder = "interference_free/ota/no_cfo/"
output_folder = "npz/"
file_register = {}
total_files = 0

with ContextManager(streamhandler_level=logging.INFO, filehandler_path=home) as contextmanager:
    
    for root, directories, _ in os.walk(storage_dir + root_folder):
        for directory in directories:
            if(directory == "bin"):
                files_in_directory = glob.glob(os.path.join(root, directory)+"/*.bin")
                subfolder = os.path.join(root, directory)+"/"
                file_register[subfolder] = files_in_directory
                total_files += len(files_in_directory)
    
    info("sys", "Found %d files in %d directories" % (total_files, len(file_register.keys())))
    
    for iteration, subfolder in enumerate(file_register.keys()):
            
        info("OS", "Diving into subfolder %s" % subfolder)
        
        # for every modulation scheme
        for mod_scheme in mod_schemes:
            
            files = [file for file in file_register[subfolder] if mod_scheme in file]
            num_files = len(files)
            
            if (num_files > 0):
                info("OS", "Analyzing samples from %s modulation" % mod_scheme)

                # find dimensionality of output
                num_samples = num_files*8 # each file = 10 sample record, discard beginning and end
                num_iq = int(read_binary(files[0]).size / 10)
                info("OS", "%d samples with %d IQ samples each" % (num_samples, num_iq))

                # pre-allocate memory
                matrix = np.zeros((num_samples, num_iq), dtype=np.complex64)
                snrs = -1 * np.ones(num_samples, dtype=float)
                labels = -1 * np.ones(num_samples, dtype=int)
                
                # for every binary file
                for i, file in enumerate(files):

                    # read data, load in dummy receiver
                    data = read_binary(file)
                    dummyReceiver.data = data
                    
                    # get SNR, label, and filter data
                    snrs[i*8:(i+1)*8] = dummyReceiver.measure_SNR()
                    labels[i*8:(i+1)*8] = mod_schemes.index(mod_scheme)
                    filtered_data = dummyReceiver.filter_data()

                    # for every chunk in a binary file (only use chunk 2-9)
                    # one chunk is equal to one measurement, observation or sample in the ML sense
                    # for this configuration every observation is t=0.001s (1ms) long
                    matrix[i*8:(i+1)*8, :] = np.stack(np.split(filtered_data, 10)[1:-1])
                
                # save data after all files have been read
                output_dir = subfolder[:-len("bin/")]+output_folder
                output_file = output_dir+mod_scheme+"_filtered.npz"

                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                np.savez(output_file, matrix=matrix, snrs=snrs, labels=labels)
                info("OS", "saved .npz data in %s" % output_file)
                
                matrix = []
                snrs = []
                labels = []

        contextmanager.step(iteration+1, len(file_register.keys()))                
        

29.03. 18:44:44  INFO 	[sys]	Started Context Manager
29.03. 18:44:47  INFO 	[sys]	Found 16000 files in 5 directories
29.03. 18:44:47  INFO 	[OS]	Diving into subfolder /storage/qnap/interference_free/ota/no_cfo/10dB/bin/
29.03. 18:44:47  INFO 	[OS]	Analyzing samples from SC_BPSK modulation
29.03. 18:44:47  INFO 	[OS]	3200 samples with 50000 IQ samples each
29.03. 18:47:09  INFO 	[OS]	saved .npz data in /storage/qnap/interference_free/ota/no_cfo/10dB/npz/SC_BPSK_filtered.npz
29.03. 18:47:09  INFO 	[OS]	Analyzing samples from SC_QPSK modulation
29.03. 18:47:09  INFO 	[OS]	3200 samples with 50000 IQ samples each
29.03. 18:49:31  INFO 	[OS]	saved .npz data in /storage/qnap/interference_free/ota/no_cfo/10dB/npz/SC_QPSK_filtered.npz
29.03. 18:49:31  INFO 	[OS]	Analyzing samples from SC_16QAM modulation
29.03. 18:49:31  INFO 	[OS]	3200 samples with 50000 IQ samples each
29.03. 18:51:52  INFO 	[OS]	saved .npz data in /storage/qnap/interference_free/ota/no_cfo/10dB/npz/SC_16QAM_filtered.npz
29.0

## Explore data

In [ ]:
with ContextManager(streamhandler_level=logging.INFO) as contextmanager:
    
    # load one example file for each modulation
    for mod_scheme in mod_schemes:
        file_register = file_registers[3][1]
        file = file_register[mod_scheme][0][0]
        info("OS", "opening %s" % file)
        data = read_binary(file)
        filtered_data = dummyReceiver.filter_data(data=data)
        dummyReceiver.visualize_samples(mode='both', data=data, filtered_data=filtered_data)
        dummyReceiver.data = data
        dummyReceiver.name="dummy"
        SNR = dummyReceiver.measure_SNR()
        info("dummy", "SNR: %5.3f dB" % SNR)

# generate some shorter files

In [16]:
npy_dir = "/storage/qnap/interference_free/cable/0db/npz/"

rows = 1000

with ContextManager(streamhandler_level=logging.DEBUG, filehandler_path=npy_dir) as contextmanager:

    files = glob.glob(npy_dir+"*filtered.npz")
    for file in files:
        
        output_file = file.rstrip(".npz")+"_short.npz"
        data = np.load(file)
        
        length = data['labels'].size
        
        randidx = np.random.randint(length, size=rows)
        matrix = data['matrix'][randidx, :]
        snrs = data['snrs'][randidx]
        labels = data['labels'][randidx]
        
        debug("OS", "opened %s" % (file))
        np.savez(output_file, matrix=matrix, snrs=snrs, labels=labels)
        debug("OS", "saved %s" % (output_file))
        matrix = []

27.03. 22:22:13  INFO 	[sys]	Started Context Manager
27.03. 22:22:36  DEBG 	[OS]	opened /storage/qnap/interference_free/cable/0db/npz/OFDM_16QAM_filtered.npz
27.03. 22:23:00  DEBG 	[OS]	saved /storage/qnap/interference_free/cable/0db/npz/OFDM_16QAM_filtered_short.npz
27.03. 22:23:20  DEBG 	[OS]	opened /storage/qnap/interference_free/cable/0db/npz/SC_BPSK_filtered.npz
27.03. 22:23:40  DEBG 	[OS]	saved /storage/qnap/interference_free/cable/0db/npz/SC_BPSK_filtered_short.npz
27.03. 22:23:55  DEBG 	[OS]	opened /storage/qnap/interference_free/cable/0db/npz/SC_16QAM_filtered.npz
27.03. 22:24:15  DEBG 	[OS]	saved /storage/qnap/interference_free/cable/0db/npz/SC_16QAM_filtered_short.npz
27.03. 22:24:34  DEBG 	[OS]	opened /storage/qnap/interference_free/cable/0db/npz/SC_64QAM_filtered.npz
27.03. 22:24:58  DEBG 	[OS]	saved /storage/qnap/interference_free/cable/0db/npz/SC_64QAM_filtered_short.npz
27.03. 22:25:14  DEBG 	[OS]	opened /storage/qnap/interference_free/cable/0db/npz/OFDM_64QAM_filtered.

In [ ]:
a = np.reshape(np.arange(15**2), (15,15))

In [ ]:
a[:,::3]

In [ ]:
np.random.randint(100, size=10)